In [5]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# --- CONFIGURATION ---
ROOT_FOLDER = "DataAnalytics"
OUTPUT_DIR = "exported_plots"

TAG_TRAIN = "train_loss"
TAG_VAL   = "valid_loss"

SMOOTHING_WEIGHT = 0.6  # 0.0 = no smoothing, 0.99 = heavy smoothing
# ---------------------

def extract_data(event_path, tag):
    """
    Returns a DataFrame if the tag exists, empty otherwise.
    """
    try:
        ea = EventAccumulator(event_path)
        ea.Reload()
        if tag in ea.Tags()['scalars']:
            scalars = ea.Scalars(tag)
            return pd.DataFrame([s.value for s in scalars], 
                                index=[s.step for s in scalars], 
                                columns=['value'])
    except Exception as e:
        print(f"  [Error reading {os.path.basename(event_path)}]: {e}")
    return pd.DataFrame()

def process_models(root_folder):
    if not os.path.exists(root_folder):
        print(f"Error: Folder '{root_folder}' not found.")
        return

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    model_dirs = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

    for model_name in model_dirs:
        print(f"--- Processing: {model_name} ---")
        model_path = os.path.join(root_folder, model_name)

        train_dfs = []
        val_dfs = []
        found_any_file = False

        # Gather Data
        for root, _, files in os.walk(model_path):
            for file in files:
                if "tfevents" in file:
                    found_any_file = True
                    full_path = os.path.join(root, file)
                    
                    t_df = extract_data(full_path, TAG_TRAIN)
                    if not t_df.empty: train_dfs.append(t_df)
                    
                    v_df = extract_data(full_path, TAG_VAL)
                    if not v_df.empty: val_dfs.append(v_df)

        if not found_any_file:
            print("  No tfevents file found.")
            continue

        # Combine and Sort
        full_train = pd.concat(train_dfs).sort_index() if train_dfs else pd.DataFrame()
        full_val = pd.concat(val_dfs).sort_index() if val_dfs else pd.DataFrame()

        if full_train.empty and full_val.empty:
            print("  No data found.")
            continue

        # --- PLOTTING ---
        plt.figure(figsize=(10, 6))
        
        # 1. Plot Train (Raw + Smoothed)
        if not full_train.empty:
            # Raw (Faint)
            plt.plot(full_train.index, full_train['value'], 
                     color='tab:blue', alpha=0.25, linewidth=1, label='_nolegend_')
            
            # Smoothed (Solid)
            # TensorBoard smoothing roughly equates to pandas ewm with alpha = 1 - weight
            smooth_train = full_train['value'].ewm(alpha=(1 - SMOOTHING_WEIGHT)).mean()
            plt.plot(full_train.index, smooth_train, 
                     color='tab:blue', linewidth=2, label='Train Loss (Smoothed)')

        # 2. Plot Validation (Raw + Smoothed)
        if not full_val.empty:
            # Raw (Faint)
            plt.plot(full_val.index, full_val['value'], 
                     color='tab:orange', alpha=0.25, linewidth=1, label='_nolegend_')
            
            # Smoothed (Solid)
            smooth_val = full_val['value'].ewm(alpha=(1 - SMOOTHING_WEIGHT)).mean()
            plt.plot(full_val.index, smooth_val, 
                     color='tab:orange', linewidth=2, label='Valid Loss (Smoothed)')

        plt.title(f"{model_name} Loss (Smoothing: {SMOOTHING_WEIGHT})")
        plt.xlabel("Steps")
        plt.ylabel("Loss")
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        save_path = os.path.join(OUTPUT_DIR, f"{model_name}_loss_smooth.png")
        plt.savefig(save_path, dpi=150)
        plt.close()
        print(f"  Saved image to {save_path}")

if __name__ == "__main__":
    process_models(ROOT_FOLDER)

--- Processing: TabTransformer ---
  Saved image to exported_plots/TabTransformer_loss_smooth.png
--- Processing: TabNet ---
  Saved image to exported_plots/TabNet_loss_smooth.png
